In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
from numpy import mean
import plotly.express as px
import matplotlib.pyplot as plt

In [5]:
gdf = gpd.read_file('data clean\estaciones_comp.shp')



In [6]:
gdf = gpd.read_file('data clean\estaciones_comp.shp')


l = gdf.columns.to_list()
l.remove('geometry')
print(l)
for i in l:

    gdf[i] = gdf[i].astype("string")

print(gdf.dtypes)
gdf.head(3)

In [ ]:

def get_df(estation):

    try:
        N = gdf[gdf['Name']== estation].index[0]

        df = pd.read_csv(gdf.loc[N,'link'], skiprows=7 )

        for i in df.index:
            df.loc[i,'fecha'] = pd.to_datetime(str(df.loc[i,'Mes']) +'/'+ str(df.loc[i,'Día']) +'/'+ str(df.loc[i,'Anio']))  

        df.set_index('fecha', inplace = True)

        df = df.drop(['Mes','Día','Anio'], axis=1)
    except:
        print('error')


    return df


def plot_station(df):

    try:
        fig = px.line(df, x=df.index ,y='Datos')
        # Add range slider
        fig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(count=1,
                             label="YTD",
                             step="year",
                             stepmode="todate"),
                        dict(count=1,
                             label="1y",
                             step="year",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            )
        )

        fig.show()
    except:
        print("no se pudo... :'v checa la estacion")





In [ ]:
est = get_df('1008')
est.head(3)

In [ ]:

plot_station(est)

In [ ]:
pd.isnull(est['Datos']).sum()

In [ ]:
'falta el {} % de los datos'.format(round((pd.isnull(est['Datos']).sum()/len(est))*100, 3))

* obtener cuantos periodos vacios
* promedio de longitud de periodos vacios
* el periodo mas largo
    




In [ ]:
np.isnan(est.iloc[0,0])

In [ ]:
est

In [ ]:


    
def periodos_foc(est, foc='faltantes'):
    dd =[]
    ondas= pd.DataFrame(columns=['inicio','final','no_dias'])

    if foc == 'faltantes':

        for i in range(0,len(est)):

            if np.isnan(est.iloc[i,0]):

                dd.append(est.iloc[i,0])
            else:
                if len(dd)>0 : 

                    rr={'inicio':est.iloc[i-len(dd)].name,
                        'final':est.iloc[i-1].name, 
                        'no_dias':len(dd) }

                    ondas = ondas.append(rr, ignore_index=True)

                dd=[]
        ondas['%']= (ondas['no_dias']/len(est))*100
                
        return ondas

    elif foc == 'completos':

        for i in range(0,len(est)):

            if ~np.isnan(est.iloc[i,0]):

                dd.append(est.iloc[i,0])
            else:
                if len(dd)>0 : 

                    rr={'inicio':est.iloc[i-len(dd)].name,
                        'final':est.iloc[i-1].name, 
                        'no_dias':len(dd) }

                    ondas = ondas.append(rr, ignore_index=True)

                dd=[]
        ondas['%']= (ondas['no_dias']/len(est))*100
        return ondas




In [ ]:
import libs.hydro as hydpy

In [ ]:
est = hydpy.get_df('1008')

In [ ]:
falt  = hydpy.periodos_foc(est, 'faltantes')
comp = hydpy.periodos_foc(est, 'completos')

In [ ]:
falt

In [ ]:
comp

In [ ]:
est.loc['1980-07-01':'2014-04-30',].plot(figsize=(25,5))

In [ ]:
len(est)

In [ ]:
est.loc['1959-07-01', 'Datos']

In [ ]:
idx  = est.index.get_loc('1980-06-30')
print(est.iloc[idx-1,0])
print(est.iloc[idx,0])
print(est.iloc[idx+1,0])

print(  (est.iloc[idx-1,0] + est.iloc[idx+1,0])/2)

In [ ]:
falt

In [ ]:
#relleno de los que solo son un dia con el promedio del dia anterior con el dia posterior

for i in falt.index:
    
    if falt.loc[i,'no_dias'] == 1: 
        print(i)
        idx  = est.index.get_loc(falt.loc[i,'inicio'])
        print(idx)
        est.iloc[idx,0] = (est.iloc[idx-1,0]+est.iloc[idx+1,0])/2
        print('+++++++++++++++++++++++')
        print(est.iloc[idx-1,0])
        print(est.iloc[idx+1,0])
        print((est.iloc[idx-1,0]+est.iloc[idx+1,0])/2)
        
        print('----------------------')
        

In [ ]:
falt2  = periodos_foc(est, 'faltantes')
comp2 = periodos_foc(est, 'completos')

In [ ]:
falt2

In [ ]:
comp2

In [ ]:
comp2[ comp2['no_dias'] == comp2['no_dias'].max()]

In [ ]:
rango = [comp2[ comp2['no_dias'] == comp2['no_dias'].max()].iloc[0,0],comp2[ comp2['no_dias'] == comp2['no_dias'].max()].iloc[0,1]]
rango

In [ ]:
complete_range = est.loc[rango[0]:rango[1],]

In [ ]:
train = complete_range.iloc[0:int(len(est)*0.75), :] 
print(len(train))

test = complete_range.iloc[int(len(est)*0.75): len(est), :]
print(len(test)) 

In [ ]:
train.plot()

In [ ]:
test.plot()

In [ ]:
print(train.tail(3))
print(test.head(3))

In [ ]:


from statsmodels.graphics.tsaplots import plot_acf 

In [ ]:
plot_acf(train['Datos'], lags = 365 , zero = False)
plt.show()